<a href="https://colab.research.google.com/github/avocadopelvis/nagamese-english/blob/main/2-training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 22.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare Your Datasets

In [ ]:
# Open the folder where you saved your prepapred datasets
%cd "/content/drive/MyDrive/MTP/nmt/"
!ls

/content/drive/MyDrive/MTP/nmt
compute-bleu.py				    naga.txt
config.yaml				    naga.txt-filtered.ng
eng.txt					    naga.txt-filtered.ng.subword
eng.txt-filtered.en			    naga.txt-filtered.ng.subword.dev
eng.txt-filtered.en.subword		    naga.txt-filtered.ng.subword.test
eng.txt-filtered.en.subword.dev		    naga.txt-filtered.ng.subword.train
eng.txt-filtered.en.subword.test	    run
eng.txt-filtered.en.subword.test.desubword  source.model
eng.txt-filtered.en.subword.train	    source.vocab
en.translated				    target.model
en.translated.desubword			    target.vocab
models					    train.log
MT-Preparation				    UN.en.translated


# Create the Training Configuration File

The following config file matches most of the recommended values for the Transformer model [Vaswani et al., 2017](https://arxiv.org/abs/1706.03762). As the current dataset is small, we reduced the following values:
* `train_steps` - for datasets with a few millions of sentences, consider using a value between 100000 and 200000, or more! Enabling the option `early_stopping` can help stop the training when there is no considerable improvement.
* `valid_steps` - 10000 can be good if the value `train_steps` is big enough.
* `warmup_steps` - obviously, its value must be less than `train_steps`. Try 4000 and 8000 values.

Refer to [OpenNMT-py training parameters](https://opennmt.net/OpenNMT-py/options/train.html) for more details. If you are interested in further explanation of the Transformer model, you can check this article, [Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/).

In [ ]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: naga.txt-filtered.ng.subword.train
        path_tgt: eng.txt-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: naga.txt-filtered.ng.subword.dev
        path_tgt: eng.txt-filtered.en.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 3000

# Default: 10000 - Run validation after n steps
valid_steps: 1000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
# [Optional] Check the content of the configuration file
!cat config.yaml

# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: naga.txt-filtered.ng.subword.train
        path_tgt: eng.txt-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: naga.txt-filtered.ng.subword.dev
        path_tgt: eng.txt-filtered.en.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 4

# D

# Build Vocabulary

For large datasets, it is not feasable to use all words/tokens found in the corpus. Instead, a specific set of vocabulary is extracted from the training dataset, usually betweeen 32k and 100k words. This is the main purpose of the vocabulary building step.

In [ ]:
# Find the number of CPUs/cores on the machine
!nproc --all

2


In [ ]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-08-22 09:48:28,076 INFO] Counter vocab from -1 samples.
[2023-08-22 09:48:28,076 INFO] n_sample=-1: Build vocab on full datasets.
[2023-08-22 09:48:30,146 INFO] Counters src: 2855
[2023-08-22 09:48:30,146 INFO] Counters tgt: 5359


From the **Runtime menu** > **Change runtime type**, make sure that the "**Hardware accelerator**" is "**GPU**".


In [ ]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9a89ab1c-a6e0-31ac-a1d3-f34c1ceae391)


In [ ]:
# Check if the GPU is visible to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14998.8125 out of: 15101.8125


# Training

Now, start training your NMT model! 🎉 🎉 🎉

In [ ]:
!rm -rf drive/MyDrive/MTP/nmt/models/

In [ ]:
# Train the NMT model
!onmt_train -config config.yaml

[2023-08-22 09:53:39,521 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-08-22 09:53:40,365 INFO] Parsed 2 corpora from -data.
[2023-08-22 09:53:40,365 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-08-22 09:53:40,397 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁laga', '.', '▁khan', '▁ke', '▁aru']
[2023-08-22 09:53:40,397 INFO] The decoder start token is: <s>
[2023-08-22 09:53:40,397 INFO] Building model...
[2023-08-22 09:53:41,073 INFO] Switching model to float32 for amp/apex_amp
[2023-08-22 09:53:41,074 INFO] Non quantized layer compute is fp16
[2023-08-22 09:53:41,341 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(2864, 512, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transfor

In [ ]:
# For error debugging try:
# !dmesg -T

# Translation

Translation Options:
* `-model` - specify the last model checkpoint name; try testing the quality of multiple checkpoints
* `-src` - the subworded test dataset, source file
* `-output` - give any file name to the new translation output file
* `-gpu` - GPU ID, usually 0 if you have one GPU. Otherwise, it will translate on CPU, which would be slower.
* `-min_length` - [optional] to avoid empty translations
* `-verbose` - [optional] if you want to print translations

Refer to [OpenNMT-py translation options](https://opennmt.net/OpenNMT-py/options/translate.html) for more details.

In [ ]:
# Translate the "subworded" source file of the test dataset
# Change the model name, if needed.
!onmt_translate -model models/model.fren_step_3000.pt -src naga.txt-filtered.ng.subword.test -output en.translated -gpu 0 -min_length 1

[2023-08-22 12:26:13,995 INFO] Loading checkpoint from models/model.fren_step_3000.pt
[2023-08-22 12:26:15,128 INFO] Loading data into the model
[2023-08-22 12:26:25,637 INFO] PRED SCORE: -0.5759, PRED PPL: 1.78 NB SENTENCES: 500


In [ ]:
# Check the first 5 lines of the translation file
!head -n 5 en.translated

▁For ▁the ▁eager ▁expect ation ▁of ▁the ▁creation ▁wait s ▁for ▁the ▁reveal ing ▁of ▁the ▁sons ▁of ▁ unless ▁it .
▁As ▁he ▁was ▁walking ▁by ▁the ▁Sea ▁of ▁Galilee , ▁he ▁saw ▁two ▁brothers , ▁Simon ▁who ▁is ▁called ▁Peter , ▁and ▁Andrew ▁his ▁brother , ▁cast ing ▁a ▁net ▁into ▁the ▁sea , ▁for ▁they ▁were ▁fishermen .
▁Le t ▁the ▁who le ▁spirit ▁of ▁gentleness ▁be ▁fulfilled ▁in ▁your ▁heart , ▁and ▁love ▁in ▁order ▁to ▁be ▁fulfilled ▁in ▁s uch ▁a ▁person . ▁Th at ▁is ▁what ▁is ▁detestable ▁things ▁of ▁the ▁offering ▁of ▁God ."
▁Hi s ▁brothers ▁therefore ▁said , ▁" Where ▁can ▁we ▁get ▁enough ▁loaves ▁of ▁olive ▁oil ? ▁Or ▁out ? ▁Or ▁naked , ▁or ▁straw , ▁or ▁in ▁the ▁sea , ▁or ▁in ▁water .
▁You ▁have ▁given ▁them ▁my ▁name , ▁so ▁that ▁you ▁are ▁ those ▁who m ▁you ▁have ▁given ▁me . ▁You ▁have ▁given ▁me , ▁and ▁they ▁have ▁given ▁me ▁to ▁everyone .


In [ ]:
# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

# Desubword the translation file
!python3 MT-Preparation/subwording/3-desubword.py target.model en.translated
# !python3 MT-Preparation/subwording/3-desubword.py source.model naga.txt-filtered.ng.subword.test

Done desubwording! Output: naga.txt-filtered.ng.subword.test.desubword


In [ ]:
# Check the first 5 lines of the desubworded translation file
!tail -n 5 en.translated.desubword

Jesus said to them, "Do not know." Jesus said to them, "Neither will I tell you by what authority I do these things. What will I do these things."
In the same way, the women should be dignified, not double-talkers. They should not drink too much wine or be greedy.
Let the one who has mercy and peace with you, have mercy on your way.
That is whatever things they are with him, and all things are through him.
He spoke many things to them about how he spoke a parable to them.


In [ ]:
# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation,
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 MT-Preparation/subwording/3-desubword.py target.model eng.txt-filtered.en.subword.test

Done desubwording! Output: eng.txt-filtered.en.subword.test.desubword


In [ ]:
# Check the first 5 lines of the desubworded reference
!tail -n 5 eng.txt-filtered.en.subword.test.desubword

Then they answered Jesus and said, "We do not know." Then Jesus said to them, "Neither will I tell you by what authority I do these things."
In the same way, their wives should be dignified, not slanderers, but sober and faithful in all things.
To all who walk according to this standard, peace and mercy be upon them, even upon the Israel of God.
He himself is before all things, and in him all things hold together.
With many parables like this he spoke the word to them, as much as they were able to understand,


# MT Evaluation

There are several MT Evaluation metrics such as BLEU, TER, METEOR, COMET, BERTScore, among others.

Here we are using BLEU. Files must be detokenized/desubworded beforehand.

In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2023-08-22 12:31:49--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2023-08-22 12:31:49 (42.6 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
# Install sacrebleu
!pip3 install sacrebleu

In [ ]:
# Evaluate the translation (without subwording)
!python3 compute-bleu.py eng.txt-filtered.en.subword.test.desubword en.translated.desubword

Reference 1st sentence: For we know that the whole creation groans and labors in pain together even now.
MTed 1st sentence: For the eager expectation of the creation waits for the revealing of the sons of unless it.
BLEU:  14.774036066234434


# More Features and Directions to Explore

Experiment with the following ideas:
* Icrease `train_steps` and see to what extent new checkpoints provide better translation, in terms of both BLEU and your human evaluation.

* Check other MT Evaluation mentrics other than BLEU such as [TER](https://github.com/mjpost/sacrebleu#ter), [WER](https://blog.machinetranslation.io/compute-wer-score/), [METEOR](https://blog.machinetranslation.io/compute-bleu-score/#meteor), [COMET](https://github.com/Unbabel/COMET), and [BERTScore](https://github.com/Tiiiger/bert_score). What are the conceptual differences between them? Is there special cases for using a specific metric?

* Continue training from the last model checkpoint using the `-train_from` option, only if the training stopped and you want to continue it. In this case, `train_steps` in the config file should be larger than the steps of the last checkpoint you train from.
```
!onmt_train -config config.yaml -train_from models/model.fren_step_3000.pt
```

* **Ensemble Decoding:** During translation, instead of adding one model/checkpoint to the `-model` argument, add multiple checkpoints. For example, try the two last checkpoints. Does it improve quality of translation? Does it affect translation seepd?

* **Averaging Models:** Try to average multiple models into one model using the [average_models.py](https://github.com/OpenNMT/OpenNMT-py/blob/master/onmt/bin/average_models.py) script, and see how this affects translation quality.
```
python3 average_models.py -models model_step_xxx.pt model_step_yyy.pt -output model_avg.pt
```
* **Release the model:** Try this command and see how it reduce the model size.
```
onmt_release_model --model "model.pt" --output "model_released.pt
```
* **Use CTranslate2:** For efficient translation, consider using [CTranslate2](https://github.com/OpenNMT/CTranslate2), a fast inference engine. Check out an [example](https://gist.github.com/ymoslem/60e1d1dc44fe006f67e130b6ad703c4b).

* **Work on low-resource languages:** Find out more details about [how to train NMT models for low-resource languages](https://blog.machinetranslation.io/low-resource-nmt/).

* **Train a multilingual model:** Find out helpful notes about [training multilingual models](https://blog.machinetranslation.io/multilingual-nmt).

* **Publish a demo:** Show off your work through a [simple demo with CTranslate2 and Streamlit](https://blog.machinetranslation.io/nmt-web-interface/).
